In [1]:
import sqlite3

db = sqlite3.connect(':memory:')  # Using an in-memory database
cur = db.cursor()

In [2]:
cur.execute('''CREATE TABLE IF NOT EXISTS Customer (
                id integer PRIMARY KEY,
                firstname varchar(255),
                lastname varchar(255) )''')
cur.execute('''CREATE TABLE IF NOT EXISTS Item (
                id integer PRIMARY KEY,
                title varchar(255),
                price decimal )''')
cur.execute('''CREATE TABLE IF NOT EXISTS BoughtItem (
                ordernumber integer PRIMARY KEY,
                customerid integer,
                itemid integer,
                price decimal,
                CONSTRAINT customerid
                    FOREIGN KEY (customerid) REFERENCES Customer(id),
                CONSTRAINT itemid
                    FOREIGN KEY (itemid) REFERENCES Item(id) )''')

In [3]:
cur.execute('''INSERT INTO Customer(firstname, lastname)
               VALUES ('Bob', 'Adams'),
                      ('Amy', 'Smith'),
                      ('Rob', 'Bennet');''')
cur.execute('''INSERT INTO Item(title, price)
               VALUES ('USB', 10.2),
                      ('Mouse', 12.23),
                      ('Monitor', 199.99);''')
cur.execute('''INSERT INTO BoughtItem(customerid, itemid, price)
               VALUES (1, 1, 10.2),
                      (1, 2, 12.23),
                      (1, 3, 199.99),
                      (2, 3, 180.00),
                      (3, 2, 11.23);''') # Discounted price 

In [4]:
cur.execute('''SELECT itemid, AVG(price) FROM BoughtItem GROUP BY itemid''')
print(cur.fetchall())

[(1, 10.2), (2, 11.73), (3, 189.995)]


In [5]:
cur.execute('''SELECT item.title, AVG(boughtitem.price) FROM BoughtItem as boughtitem INNER JOIN Item as item on (item.id = boughtitem.itemid) GROUP BY boughtitem.itemid''')
print(cur.fetchall())

[('USB', 10.2), ('Mouse', 11.73), ('Monitor', 189.995)]


In [7]:
cur.execute('''SELECT customer.firstname, SUM(boughtitem.price) FROM BoughtItem as boughtitem INNER JOIN Customer as customer on (customer.id = boughtitem.customerid) GROUP BY customer.firstname''')
print(cur.fetchall())

[('Amy', 180), ('Bob', 222.42000000000002), ('Rob', 11.23)]


In [6]:
cur.execute('''EXPLAIN QUERY PLAN SELECT customer.firstname, item.title, 
...                item.price, boughtitem.price FROM BoughtItem as boughtitem
...                INNER JOIN Customer as customer on (customer.id = boughtitem.customerid)
...                INNER JOIN Item as item on (item.id = boughtitem.itemid)''')
...
print(cur.fetchall())

[(0, 0, 0, 'SCAN TABLE BoughtItem AS boughtitem'), (0, 1, 1, 'SEARCH TABLE Customer AS customer USING INTEGER PRIMARY KEY (rowid=?)'), (0, 2, 2, 'SEARCH TABLE Item AS item USING INTEGER PRIMARY KEY (rowid=?)')]


In [7]:
!pip install pymongo

     |████████████████████████████████| 358kB 4.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

# Note: This database is not created until it is populated by some data
db = client["example_database"]

customers = db["customers"]
items = db["items"]

customers_data = [{ "firstname": "Bob", "lastname": "Adams" },
                  { "firstname": "Amy", "lastname": "Smith" },
                  { "firstname": "Rob", "lastname": "Bennet" },]
items_data = [{ "title": "USB", "price": 10.2 },
              { "title": "Mouse", "price": 12.23 },
              { "title": "Monitor", "price": 199.99 },]

customers.insert_many(customers_data)
items.insert_many(items_data)

ModuleNotFoundError: No module named 'pymongo'